# NLP POS Substitution for Poems



Part 1: NLP POS Substitution for Poems

Find a couple of  poems from 2 poets [for example Robert Bly and Emily Dickinson ]


Home Work

1. Nice printing etc, see slides in Slack [edited to list details below]
    1. Print it out on the console so you can read and assist in evaluating the outcome of the transposition. Is the transposed poem a good poem? Does it make sense? Was the semantic similarity a good enough way of finding a suitable substitution?  
    2. Do I have to run the notebook again even though I have run it before and have produced results? If so, write defensive code to avoid re-running the parts of the notebook.
        1. scraped data
        2. json representation : parts of speech, author, poem name, poem text, url
        3. text files
    3. When I want to substitute/transpose the POS, Which POS should I use from the other poet?
        1. start with a poem from poet A
        2. find POS for poet A
        3. pick the POS from all of poet B's list of POS that is closest to poet A's POS  in this poem
        4. transpose it
        5. do this for all poems for poet A, then do this for all poems

    4. Analyze the similarity of poems
        1. as-is between poets
        2. similarity when you swap nouns
        3. similarity when you swap verbs
        4. similarity when you swap adjectives
        5. similarity when you swap adverbs
        6. similarity when you swap all


In [ ]:
poet1 = "William Blake"
poet2 = "Robert Frost"
store_loc = "/content/drive/MyDrive/poets2/"

#Part 1 : POS Substitutions for Poetry

In [ ]:
import pickle

try:
  with open('notebook_state.pkl', 'rb') as f:
    notebook_state = pickle.load(f)
except FileNotFoundError:
  notebook_state = {'has_run': False}

if notebook_state.get('has_run', False):
  pass
else:
  !pip install --upgrade setuptools pip numpy bertopic pyLDAvis contractions sentence-transformers
  notebook_state['has_run'] = True

with open('notebook_state.pkl', 'wb') as f:
  pickle.dump(notebook_state, f)


Restart the runtime and run all. The runtime must be restarted to ensure the newly installed versions are in place. The above code has defensive code to ensure the libraries are not installed again, so when the cells are re-run they will not be installed again.

In [ ]:
import requests
import pandas as pd
import bs4 as bs
import urllib.request
import json
import contractions
import re
import nltk
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer, util
from nltk.corpus import stopwords
from transformers import pipeline
from spacy import displacy
from bertopic import BERTopic
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import pyLDAvis
import gensim
import spacy
import gensim.corpora as corpora
from gensim.models import CoherenceModel
import pyLDAvis.gensim_models as gensimvis
from gensim.utils import simple_preprocess
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.utils import simple_preprocess

Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/google/rpc/__init__.py:20: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.rpc')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  pkg_resources.declare_namespace(__name__)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2350: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(parent)


Need to restart to make this work. See if you can dynamically do this somehow

In [ ]:
import warnings
warnings.filterwarnings('ignore', 'Calling')
warnings.filterwarnings('ignore', 'For a faster implementation')
warnings.filterwarnings('ignore', 'In')
warnings.filterwarnings('ignore', 'SelectableGroups dict interface')
warnings.filterwarnings('ignore', 'np')
warnings.filterwarnings('ignore', 'SelectableGroups')

In [ ]:
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Scrape 2 poems from 2 Poets

In [ ]:
def clean_txt(text):
    text=text.replace(u'\xa0', u' ').replace(r'\r',u'\n').replace(r'\n', ' ').replace('\n', ' ').replace('\t', ' ')
    text = contractions.fix(text)
    text = text.strip().lower()
    text = re.sub(r"'", ' ', text)
    filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    translate_dict = dict((i, " ") for i in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)
    text = ' '.join([w for w in text.split() if len(w)>1])
    text = re.sub(' +', ' ', text)
    text = ''.join(text)
    return text

def scrapePoemFromURL(poem_url_list):
    poems_results=[]
    for url in poem_url_list:
        header = {'User-Agent':'Mozilla/5.0'}
        requests = urllib.request.Request(url,headers=header)
        requestread = urllib.request.urlopen(requests).read()
        soup = bs.BeautifulSoup(requestread,'html.parser')
        poem_for_human = soup.find_all('div', class_="poem_body")[0].text.replace("© by owner. provided at no charge for educational purposes", "")
        poem_for_machine = (clean_txt(poem_for_human))
        poem_title = soup.find_all('h1')[0].text
        poems_results.append([poem_title, poem_for_machine, poem_for_human])
    return poems_results

In [ ]:
def get_poem_url_json(poet):
    site = "https://allpoetry.com"
    search = "https://allpoetry.com/items/read_by/"
    page = 1
    URL_list = list()
    while len(URL_list) < 20:
        url = search + poet + "?page=" + str(page)
        response = requests.get(url)
        soup = bs.BeautifulSoup(response.text, 'lxml')
        poem_list = soup.find_all('a', href=True, class_='nocolor fn')
        for i in range(len(poem_list)):
            #print(poem_list[i]['href'])
            URL_list.append(site + poem_list[i]['href'])
            if len(URL_list) == 20:
                break
        page += 1
    return {"poet":poet, "URL":URL_list}

In [ ]:
pd_title_list = ["Title", "Poem", "Poem_for_Human"]

In [ ]:
def getPoem(file_name, URLs):
    df = pd.DataFrame()
    if os.path.exists(file_name):
        df = pd.read_csv(file_name)
    else:
        os.makedirs(os.path.dirname(file_name), exist_ok=True)
        df = pd.DataFrame(scrapePoemFromURL(URLs), columns=pd_title_list)
        df.to_csv(file_name)
    return df

If the poem file exist, avoid re-scrape it. Else, scrape it.

In [ ]:
#William Blake
file_name = store_loc + poet1.replace(" ", "_") + ".csv"
WS_df = getPoem(file_name, get_poem_url_json(poet1)['URL'])

#Robert Frost
file_name = store_loc + poet2.replace(" ", "_") + ".csv"
JWG_df = getPoem(file_name, get_poem_url_json(poet2)['URL'])

## POS in all poem in JSON

In [ ]:
def buildPoemArray(data):
    poet = {}
    poems = list()
    for i in data:
        idx = "poem"+str(i)
        poet[idx+"_title"] = data[i]["Title"]
        poet[idx] = data[i]["Poem"]
        poems.append(idx)
    poet['poems_array'] = poems
    return poet

In [ ]:
# def extractAllPOS(poet):
#     global_vrb = set()
#     global_nns = set()
#     global_adj = set()
#     global_advrb = set()
#     for key in poet['poems_array']:
#         text = poet[key]
#         word_list = word_tokenize(text)
#         pos_value = nltk.pos_tag(word_list)
#         print(pos_value)
#         poet["pos_"+key] = pos_value

#         vrb = set([word for (word, pos) in pos_value if (pos.startswith('VB'))])
#         nns = set([word for (word, pos) in pos_value if (pos.startswith('NN'))])
#         adj = set([word for (word, pos) in pos_value if (pos.startswith('JJ'))])
#         adverb = set([word for (word, pos) in pos_value if (pos.startswith('ADV'))])

#         poet["verbs_"+key] = list(vrb)
#         poet["nouns_"+key] = list(nns)
#         poet["adjectives_"+key] = list(adj)
#         poet["adverb_"+key] = list(adj)

#         global_vrb = set.union(global_vrb, vrb)
#         global_nns = set.union(global_nns, nns)
#         global_adj = set.union(global_adj, adj)
#         global_advrb = set.union(global_adj, adj)

#     poet["all_verbs"] = list(global_vrb)
#     poet["all_nouns"] = list(global_nns)
#     poet["all_adjectives"] = list(global_adj)
#     poet["all_adverbs"] = list(global_adj)


POS with adverbs

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

def extractAllPOS(poet):
    global_verbs = set()
    global_nouns = set()
    global_adjectives = set()
    global_adverbs = set()

    for key in poet['poems_array']:
        text = poet[key]
        doc = nlp(text)

        pos_value = [(token.text, token.pos_) for token in doc]
        poet["pos_"+key] = pos_value

        verbs = set([word for (word, pos) in pos_value if pos.startswith('VERB')])
        nouns = set([word for (word, pos) in pos_value if pos.startswith('NOUN')])
        adjectives = set([word for (word, pos) in pos_value if pos.startswith('ADJ')])
        adverbs = set([word for (word, pos) in pos_value if pos.startswith('ADV')])

        poet["verbs_"+key] = list(verbs)
        poet["nouns_"+key] = list(nouns)
        poet["adjectives_"+key] = list(adjectives)
        poet["adverbs_"+key] = list(adverbs)

        global_verbs = set.union(global_verbs, verbs)
        global_nouns = set.union(global_nouns, nouns)
        global_adjectives = set.union(global_adjectives, adjectives)
        global_adverbs = set.union(global_adverbs, adverbs)

    poet["all_verbs"] = list(global_verbs)
    poet["all_nouns"] = list(global_nouns)
    poet["all_adjectives"] = list(global_adjectives)
    poet["all_adverbs"] = list(global_adverbs)

In [ ]:
def getPOS(file_name, df):
    pos = ""
    if os.path.exists(file_name):
        with open(file_name, 'r') as file:
            pos = json.load(file)
    else:
        pos = buildPoemArray(json.loads(df.to_json(orient="index")))
        extractAllPOS(pos)
        with open(file_name, 'w') as file:
            json.dump(pos, file)
    return pos

In [ ]:
def getJSON(file_name, df_name_1, df_1, df_name_2, df_2):
    pos_json = ""
    if os.path.exists(file_name):
        with open(file_name, 'r') as file:
            pos_json = json.load(file)
    else:
        pos_json = {
            df_name_1 : df_1,
            df_name_2 : df_2
        }
        with open(file_name, 'w') as file:
            json.dump(pos_json, file)
    return pos_json

If the POS JSON file exist, avoid re-POS it. Else, POS it.

In [ ]:
file_name = store_loc + poet1.replace(" ", "_") + ".json"
WS_pos = getPOS(file_name, WS_df)

In [ ]:
# file_name = store_loc + poet1.replace(" ", "_") + ".json"
# WS_pos = getPOS(file_name, WS_df)
file_name = store_loc + poet2.replace(" ", "_") + ".json"
JWG_pos = getPOS(file_name, JWG_df)
file_name = store_loc + poet1.replace(" ", "_") + "-" + poet2.replace(" ", "_") + ".json"
poets_json = getJSON(file_name, poet1.replace(" ", "_"), WS_pos, poet2.replace(" ", "_"), JWG_pos)

In [ ]:
#combining POS for both the poets
# poets_json

## Transpose verbs, Adjectives and nouns

In [ ]:
def getSemanticSimilarity(p1, p2, model):
    embedding1 = model.encode(p1, convert_to_tensor=True)
    embedding2 = model.encode(p2, convert_to_tensor=True)
    return util.pytorch_cos_sim(embedding1, embedding2).item()

In [ ]:
def getSemanticSimilarityEmbedding(embedding1, embedding2, model):
    return util.pytorch_cos_sim(embedding1, embedding2).item()

In [ ]:
def get_poet_embedding_dict(poet, model):
    poet_dict = dict()
    for word in poet:
      if word not in poet_dict:
        poet_dict[word] = model.encode(word, convert_to_tensor=True)
    return poet_dict

In [ ]:
def swapWords(poet1, poet2, poem, cs_threshold):
    model = SentenceTransformer('stsb-roberta-large')
    import time
    start = time.time()
    poet1_dict = get_poet_embedding_dict(poet1, model)
    end = time.time()
    start = time.time()
    poet2_dict = get_poet_embedding_dict(poet2, model)
    end = time.time()

    swapped_words_table = []

    # cs_threshold = 0.6
    new_poem = poem
    for word1 in poet1:
        max_word = ""
        max_sim = 0
        for word2 in poet2:
            if word1 == word2:
                continue
            sim = getSemanticSimilarityEmbedding(poet1_dict[word1], poet2_dict[word2], model)
            if max_sim < sim:
                max_sim = sim
                max_word = word2
        if max_sim > cs_threshold:
            new_poem = new_poem.replace(word1, max_word)
            swapped_words_table.append([word1, max_word])
    return new_poem, swapped_words_table

## replace poet A POS with poet B POS based in semantic similarity

In [ ]:
def newPoem(file_name, poet_1_pos, poet_2_pos, poem, index, cs_threshold):
    new_poem = ""
    swapped_table = []
    name, extension = os.path.splitext(file_name)
    new_file_name = f"{name}_swapped_words{extension}"
    if os.path.exists(file_name):
        with open(file_name, 'r') as file:
            new_poem = file.read()
        with open(new_file_name, 'r') as file:
            swapped_table = file.read()
    else:
        new_poem, verbs_swapped = swapWords(poet_1_pos['verbs_poem' + str(index)], poet_2_pos['all_verbs'], poem[index].lower(), cs_threshold)
        new_poem, adjectives_swapped = swapWords(poet_1_pos['adjectives_poem'+ str(index)], poet_2_pos['all_adjectives'], new_poem, cs_threshold)
        new_poem, nouns_swapped = swapWords(poet_1_pos['nouns_poem'+ str(index)], poet_2_pos['all_nouns'], new_poem, cs_threshold)
        swapped_table.extend(verbs_swapped)
        swapped_table.extend(adjectives_swapped)
        swapped_table.extend(nouns_swapped)
        with open(file_name, 'w') as file:
            file.write(new_poem)
        with open(new_file_name, 'w') as file:
            file.write(str(swapped_table))
    return new_poem, swapped_table

If the new poem text file exist, avoid replace it. Else, replace it.

In [ ]:
for i in range(0):
    print(i)
    file_name = store_loc + poet1.replace(" ", "_") + '-' + poet2.replace(" ", "_") + '-' + WS_df.Title[i].replace('\'','').replace('-','').replace(',','').replace(':','').replace('  ', ' ').replace(' ', '_') + '.txt'
    newPoem(file_name, WS_pos, JWG_pos, WS_df.Peom_for_Human, i, 0.6)

In [ ]:
for i in range(0):
    file_name = store_loc + poet2.replace(" ", "_") + '-' + poet1.replace(" ", "_") + '-' + JWG_df.Title[i].replace('\'','').replace('-','').replace(',','').replace(':','').replace('  ', ' ').replace(' ', '_') + '.txt'
    newPoem(file_name, JWG_pos, WS_pos, JWG_df.Peom_for_Human, i, 0.6)

In [ ]:
cs_threshold = 0.6

In [ ]:
file_name = store_loc + poet1.replace(" ", "_") + '-' + poet2.replace(" ", "_") + '-poem0.txt'
WS_p1_new, swap_table_WS_p1 = newPoem(file_name, WS_pos, JWG_pos, WS_df.Poem_for_Human, 0, cs_threshold)

In [ ]:
file_name = store_loc + poet1.replace(" ", "_") + '-' + poet2.replace(" ", "_") + '-poem1.txt'
WS_p2_new, swap_table_WS_p2 = newPoem(file_name, WS_pos, JWG_pos, WS_df.Poem_for_Human, 1, cs_threshold)

In [ ]:
file_name = store_loc + poet2.replace(" ", "_") + '-' + poet1.replace(" ", "_") + '-poem0.txt'
JWG_p1_new, swap_table_JWG_p1 = newPoem(file_name, JWG_pos, WS_pos, JWG_df.Poem_for_Human, 0, cs_threshold)

In [ ]:
file_name = store_loc + poet2.replace(" ", "_") + '-' + poet1.replace(" ", "_") + '-poem1.txt'
JWG_p2_new, swap_table_JWG_p2 = newPoem(file_name, JWG_pos, WS_pos, JWG_df.Poem_for_Human, 1, cs_threshold)

## Similarity of poems

Before Swapping: Poem 1

In [ ]:
getSemanticSimilarity(WS_df.Poem[0], JWG_df.Poem[0], SentenceTransformer('stsb-roberta-large'))

0.3649998903274536

After Swapping: Poem 1

In [ ]:
getSemanticSimilarity(WS_p1_new, JWG_df.Poem[0], SentenceTransformer('stsb-roberta-large'))

0.33516478538513184

Before Swapping: Poem 2

In [ ]:
getSemanticSimilarity(WS_df.Poem[1], JWG_df.Poem[1], SentenceTransformer('stsb-roberta-large'))

0.17570196092128754

After Swapping: Poem 2

In [ ]:
getSemanticSimilarity(WS_p2_new, JWG_df.Poem[1], SentenceTransformer('stsb-roberta-large'))

0.10183341056108475

##Comparing similarity of old and new peoms

In [ ]:
similarity = pd.DataFrame(columns=['William Blake', 'Robert Frost', 'Noun Swap William Blake', 'Noun Swap Robert Frost', 'Adjective Swap William Blake', 'Adjective Swap Robert Frost',
                                   'Verb Swap William Blake', 'Verb Swap Robert Frost', 'Adverb Swap William Blake', 'Adverb Swap Robert Frost'])

In [ ]:
similarity.at[0, "William Blake"] = getSemanticSimilarity(WS_df.Poem[0], WS_p1_new, SentenceTransformer('stsb-roberta-large'))

similarity.at[1, "William Blake"] = getSemanticSimilarity(WS_df.Poem[1], WS_p2_new, SentenceTransformer('stsb-roberta-large'))

similarity.at[0, "Robert Frost"] = getSemanticSimilarity(JWG_df.Poem[0], JWG_p1_new, SentenceTransformer('stsb-roberta-large'))

similarity.at[1, "Robert Frost"] = getSemanticSimilarity(JWG_df.Poem[1], JWG_p2_new, SentenceTransformer('stsb-roberta-large'))

##Similarity of poems after swapping nouns

In [ ]:
new_poem_WS_noun_swap, swap_table = swapWords(WS_pos['verbs_poem' + str(0)], JWG_pos['all_verbs'], WS_df.Poem_for_Human[0].lower(), 0.6)
similarity.at[0, "Noun Swap William Blake"] = getSemanticSimilarity(WS_df.Poem[0], new_poem_WS_noun_swap, SentenceTransformer('stsb-roberta-large'))

new_poem_WS_noun_swap, swap_table = swapWords(WS_pos['verbs_poem' + str(1)], JWG_pos['all_verbs'], WS_df.Poem_for_Human[1].lower(), 0.6)
similarity.at[1, "Noun Swap William Blake"] = getSemanticSimilarity(WS_df.Poem[0], new_poem_WS_noun_swap, SentenceTransformer('stsb-roberta-large'))

new_poem_JW_noun_swap, swap_table = swapWords(JWG_pos['verbs_poem' + str(0)], WS_pos['all_verbs'], JWG_df.Poem_for_Human[0].lower(), 0.6)
similarity.at[0, "Noun Swap Robert Frost"] = getSemanticSimilarity(WS_df.Poem[0], new_poem_JW_noun_swap, SentenceTransformer('stsb-roberta-large'))

new_poem_JW_noun_swap, swap_table = swapWords(JWG_pos['verbs_poem' + str(1)], WS_pos['all_verbs'], JWG_df.Poem_for_Human[1].lower(), 0.6)
similarity.at[1, "Noun Swap Robert Frost"] = getSemanticSimilarity(WS_df.Poem[0], new_poem_JW_noun_swap, SentenceTransformer('stsb-roberta-large'))

##Similarity of poems after swapping adjectives

In [ ]:
new_poem_WS_noun_swap, swap_table = swapWords(WS_pos['adjectives_poem' + str(0)], JWG_pos['all_adjectives'], WS_df.Poem_for_Human[0].lower(), 0.6)
similarity.at[0, "Adjective Swap William Blake"] = getSemanticSimilarity(WS_df.Poem[0], new_poem_WS_noun_swap, SentenceTransformer('stsb-roberta-large'))

new_poem_WS_noun_swap, swap_table = swapWords(WS_pos['adjectives_poem' + str(1)], JWG_pos['all_adjectives'], WS_df.Poem_for_Human[1].lower(), 0.6)
similarity.at[1, "Adjective Swap William Blake"] = getSemanticSimilarity(WS_df.Poem[0], new_poem_WS_noun_swap, SentenceTransformer('stsb-roberta-large'))

new_poem_JW_noun_swap, swap_table = swapWords(JWG_pos['adjectives_poem' + str(0)], WS_pos['all_adjectives'], JWG_df.Poem_for_Human[0].lower(), 0.6)
similarity.at[0, "Adjective Swap Robert Frost"] = getSemanticSimilarity(WS_df.Poem[0], new_poem_JW_noun_swap, SentenceTransformer('stsb-roberta-large'))

new_poem_JW_noun_swap, swap_table = swapWords(JWG_pos['adjectives_poem' + str(1)], WS_pos['all_adjectives'], JWG_df.Poem_for_Human[1].lower(), 0.6)
similarity.at[1, "Adjective Swap Robert Frost"] = getSemanticSimilarity(WS_df.Poem[0], new_poem_JW_noun_swap, SentenceTransformer('stsb-roberta-large'))

##Similarity of poems after swapping verbs

In [ ]:
new_poem_WS_noun_swap, swap_table = swapWords(WS_pos['verbs_poem' + str(0)], JWG_pos['all_verbs'], WS_df.Poem_for_Human[0].lower(), 0.6)
similarity.at[0, "Verb Swap William Blake"] = getSemanticSimilarity(WS_df.Poem[0], new_poem_WS_noun_swap, SentenceTransformer('stsb-roberta-large'))

new_poem_WS_noun_swap, swap_table = swapWords(WS_pos['verbs_poem' + str(1)], JWG_pos['all_verbs'], WS_df.Poem_for_Human[1].lower(), 0.6)
similarity.at[1, "Verb Swap William Blake"] = getSemanticSimilarity(WS_df.Poem[0], new_poem_WS_noun_swap, SentenceTransformer('stsb-roberta-large'))

new_poem_JW_noun_swap, swap_table = swapWords(JWG_pos['verbs_poem' + str(0)], WS_pos['all_verbs'], JWG_df.Poem_for_Human[0].lower(), 0.6)
similarity.at[0, "Verb Swap Robert Frost"] = getSemanticSimilarity(WS_df.Poem[0], new_poem_JW_noun_swap, SentenceTransformer('stsb-roberta-large'))

new_poem_JW_noun_swap, swap_table = swapWords(JWG_pos['verbs_poem' + str(1)], WS_pos['all_verbs'], JWG_df.Poem_for_Human[1].lower(), 0.6)
similarity.at[1, "Verb Swap Robert Frost"] = getSemanticSimilarity(WS_df.Poem[0], new_poem_JW_noun_swap, SentenceTransformer('stsb-roberta-large'))

##Similarity of poems after swapping adverbs

In [ ]:
new_poem_WS_noun_swap, swap_table = swapWords(WS_pos['adverbs_poem' + str(0)], JWG_pos['all_adverbs'], WS_df.Poem_for_Human[0].lower(), 0.6)
similarity.at[0, "Adverb Swap William Blake"] = getSemanticSimilarity(WS_df.Poem[0], new_poem_WS_noun_swap, SentenceTransformer('stsb-roberta-large'))

new_poem_WS_noun_swap, swap_table = swapWords(WS_pos['adverbs_poem' + str(1)], JWG_pos['all_adverbs'], WS_df.Poem_for_Human[1].lower(), 0.6)
similarity.at[1, "Adverb Swap William Blake"] = getSemanticSimilarity(WS_df.Poem[0], new_poem_WS_noun_swap, SentenceTransformer('stsb-roberta-large'))

new_poem_JW_noun_swap, swap_table = swapWords(JWG_pos['adverbs_poem' + str(0)], WS_pos['all_adverbs'], JWG_df.Poem_for_Human[0].lower(), 0.6)
similarity.at[0, "Adverb Swap Robert Frost"] = getSemanticSimilarity(WS_df.Poem[0], new_poem_JW_noun_swap, SentenceTransformer('stsb-roberta-large'))

new_poem_JW_noun_swap, swap_table = swapWords(JWG_pos['adverbs_poem' + str(1)], WS_pos['all_adverbs'], JWG_df.Poem_for_Human[1].lower(), 0.6)
similarity.at[1, "Adverb Swap Robert Frost"] = getSemanticSimilarity(WS_df.Poem[0], new_poem_JW_noun_swap, SentenceTransformer('stsb-roberta-large'))

In [ ]:
similarity

,William Blake,Robert Frost,Noun Swap William Blake,Noun Swap Robert Frost,Adjective Swap William Blake,Adjective Swap Robert Frost,Verb Swap William Blake,Verb Swap Robert Frost,Adverb Swap William Blake,Adverb Swap Robert Frost
0,0.948789,0.646165,0.949886,0.296189,0.954228,0.325656,0.949886,0.296189,0.947971,0.313904
1,0.886658,0.710712,0.413494,0.286338,0.397427,0.260176,0.413494,0.286338,0.407946,0.27228


## observe similarity after changing cosine threshold

In [ ]:
#with 0.8 cosine threshold
cs_threshold = 0.8

file_name = store_loc + poet1.replace(" ", "_") + '-' + poet2.replace(" ", "_") + '-poem0-0.8_Threshold.txt'
WS_p1_ch_threshold, swap_table_WS_p1 = newPoem(file_name, WS_pos, JWG_pos, WS_df.Poem_for_Human, 0, cs_threshold)

getSemanticSimilarity(WS_df.Poem[0], WS_p1_ch_threshold, SentenceTransformer('stsb-roberta-large'))

0.9553618431091309

In [ ]:
#with 0.4 cosine threshold
cs_threshold = 0.4

file_name = store_loc + poet1.replace(" ", "_") + '-' + poet2.replace(" ", "_") + '-poem0-0.4_Threshold.txt'
WS_p1_ch_threshold, swap_table_WS_p1 = newPoem(file_name, WS_pos, JWG_pos, WS_df.Poem_for_Human, 0, cs_threshold)

getSemanticSimilarity(WS_df.Poem[0], WS_p1_ch_threshold, SentenceTransformer('stsb-roberta-large'))

0.7416212558746338

## In human readable format

In [ ]:
print ('Original poem 1 by William Blake:\n\n', WS_df.Poem_for_Human[0])
print("\n\nSwapped words [<old word>, <new word>]:\n", swap_table_WS_p1)

Original poem 1 by William Blake:

 O Rose, thou art sick! 
The invisible worm
That flies in the night,
In the howling storm,
Has found out thy bed
Of crimson joy:
And his dark secret love
Does thy life destroy.     


Swapped words [<old word>, <new word>]:
 [['does', 'do'], ['flies', 'flung'], ['destroy', 'shattering'], ['found', 'find'], ['rose', 'rises'], ['howling', 'had'], ['secret', 'unknown'], ['sick', 'saddest'], ['invisible', 'unknown'], ['thou', 'heavy'], ['dark', 'darkest'], ['love', 'lovers'], ['life', 'length'], ['joy', 'happiness'], ['worm', 'tickles'], ['bed', 'house'], ['night', 'dark'], ['storm', 'storms'], ['art', 'wall']]


In [ ]:
print(WS_p1_new)

o rose, thou art sick! 
the unknown worm
that flies in the dark,
in the howling storms,
has find out thy bed
of crimson happiness:
and his darkest secret lovers
do thy life destroy.     


In [ ]:
print ('Original poem 2 by William Blake:\n\n', WS_df.Poem_for_Human[1])
print("\n\nSwapped words [<old word>, <new word>]:\n", swap_table_WS_p2)

Original poem 2 by William Blake:

 Never seek to tell thy love,  
Love that never told can be;
For the gentle wind does move
Silently, invisibly.

I told my love, I told my love,
I told her all my heart;
Trembling, cold, in ghastly fears,
Ah! she did depart!

Soon as she was gone from me,
A traveller came by,
Silently, invisibly 
He took her with a sigh.
     


Swapped words [<old word>, <new word>]:
 [['tell', 'telling'], ['told', 'tell'], ['move', 'moves'], ['took', 'taking'], ['gone', 'went'], ['came', 'comes'], ['seek', 'seeking'], ['depart', 'went'], ['fears', 'fright'], ['love', 'lovers'], ['cold', 'winter'], ['traveller', 'traveler']]


In [ ]:
print(WS_p2_new)

never seeking to telling thy lovers,  
lovers that never tell can be;
for the gentle wind does moves
silently, invisibly.

i tell my lovers, i tell my lovers,
i tell her all my heart;
trembling, winter, in ghastly fright,
ah! she did went!

soon as she was went from me,
a traveler comes by,
silently, invisibly 
he taking her with a sigh.
     


In [ ]:
import re

print ('Original poem 1 by Robert Frost:\n\n', re.sub(r'[\r\n\t]+', '\n',JWG_df.Poem_for_Human[0]))
print("\n\nSwapped words [<old word>, <new word>]:\n", swap_table_JWG_p1)

Original poem 1 by Robert Frost:

 
 
 
  Some say the world will end in fire,
  Some say in ice.
  From what I've tasted of desire
  I hold with those who favor fire.
  But if it had to perish twice,
  I think I know enough of hate
  To know that for destruction ice
  Is also great
  And would suffice.
 
 
 
 
 
 
Composition date is unknown - The lyrical form of this poem is abaabcbcb.
     


Swapped words [<old word>, <new word>]:
 [['say', 'said'], ['think', 'thought'], ['hold', 'clasp'], ['end', 'done'], ['had', 'have'], ['know', 'known'], ['perish', 'hurt'], ['unknown', 'invisible'], ['enough', 'such'], ['great', 'good'], ['desire', 'desires'], ['destruction', 'destroyor'], ['fire', 'fiery'], ['ice', 'snow'], ['hate', 'wrath']]


In [ ]:
print(JWG_p1_new)


	 

	 

	  some said the world will done in fiery,
	  some said in snow.
	  from what i've tasted of desires
	  i clasp with those who favor fiery.
	  but if it have to hurt twsnow,
	  i thought i known such of wrath
	  to known that for destroyor snow
	  is also good
	  and would suffsnow.

	 

	 

	 

	 

	 

	 

	composition date is invisiblen - the lyrical form of this poem is abaabcbcb.
     


In [ ]:
print ('Original poem 2 by Robert Frost ',  re.sub(r'[\r\n\t]+', '\n',JWG_df.Poem_for_Human[1]))
print("\n\nSwapped words [<old word>, <new word>]:\n", swap_table_JWG_p1)

Original poem 2 by Robert Frost  Nature's first green is gold, 
Her hardest hue to hold.
Her early leaf's a flower;
But only so an hour.
Then leaf subsides to leaf.
So Eden sank to grief,
So dawn goes down to day.
Nothing gold can stay.     


Swapped words [<old word>, <new word>]:
 [['say', 'said'], ['think', 'thought'], ['hold', 'clasp'], ['end', 'done'], ['had', 'have'], ['know', 'known'], ['perish', 'hurt'], ['unknown', 'invisible'], ['enough', 'such'], ['great', 'good'], ['desire', 'desires'], ['destruction', 'destroyor'], ['fire', 'fiery'], ['ice', 'snow'], ['hate', 'wrath']]


In [ ]:
print(JWG_p2_new)

nature's first green is gems, 
her strongest hue to clasp.

her early leaves's a flowers;
but only so an hour.

then leaves subsides to leaves.
so eden sank to tears,

so morning went down to morning.
nothing gems can keeps.     


# Part 2 : Topic Modeling and Summarization

## Summarization

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
summarizer = pipeline("summarization", model="t5-base", tokenizer="t5-base", framework="tf")

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
def removeQuotes(mydata):
    new_data = mydata.strip('“”')
    return new_data
def SummaryOfPoem(poet):
  count = 0
  for key in poet['poems_array']:
    count += 1;
    text = poet[key]
    text = removeQuotes(text)
    summary_text = summarizer(text, max_length=50, min_length=5, do_sample=False)[0]['summary_text']
    poet["summary_"+key] = summary_text

  return poet

In [ ]:
def getSummary(file_name, df):
    summary = ""
    if os.path.exists(file_name):
        with open(file_name, 'r') as file:
            summary = json.load(file)
    else:
        summary = SummaryOfPoem(buildPoemArray(json.loads(df.to_json(orient="index"))))
        with open(file_name, 'w') as file:
            json.dump(summary, file)
    return summary

In [ ]:
file_name = store_loc + poet1.replace(" ", "_") + "-Summary.json"
WS_summary = getSummary(file_name, WS_df)
file_name = store_loc + poet2.replace(" ", "_") + "-Summary.json"
JWG_summary = getSummary(file_name, JWG_df)

Your max_length is set to 50, but your input_length is only 48. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=24)
Your max_length is set to 50, but your input_length is only 39. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=19)
Token indices sequence length is longer than the specified maximum sequence length for this model (698 > 512). Running this sequence through the model will result in indexing errors


## modifying library used for cosine similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine

def getSemanticSimilarity_diff(embedding1, embedding2, model):
    if model == 1:
        return cosine_similarity([embedding1], [embedding2])[0][0]
    else:
        similarity = 1 - cosine(embedding1, embedding2)
        return similarity


In [ ]:
def swapWords_new(poet1, poet2, poem, cs_threshold, algo):
    model = SentenceTransformer('stsb-roberta-large')
    import time
    start = time.time()
    poet1_dict = get_poet_embedding_dict(poet1, model)
    end = time.time()
    start = time.time()
    poet2_dict = get_poet_embedding_dict(poet2, model)
    end = time.time()

    swapped_words_table = []

    # cs_threshold = 0.6
    new_poem = poem
    for word1 in poet1:
        max_word = ""
        max_sim = 0
        for word2 in poet2:
            if word1 == word2:
                continue
            sim = getSemanticSimilarity_diff(poet1_dict[word1], poet2_dict[word2], algo)
            if max_sim < sim:
                max_sim = sim
                max_word = word2
        if max_sim > cs_threshold:
            new_poem = new_poem.replace(word1, max_word)
            swapped_words_table.append([word1, max_word])
    return new_poem, swapped_words_table

In [ ]:
def newPoem_new(poet_1_pos, poet_2_pos, poem, index, cs_threshold, algo):
    new_poem = ""
    swapped_table = []
    new_poem, verbs_swapped = swapWords_new(poet_1_pos['verbs_poem' + str(index)], poet_2_pos['all_verbs'], poem[index].lower(), cs_threshold, algo)
    new_poem, adjectives_swapped = swapWords_new(poet_1_pos['adjectives_poem'+ str(index)], poet_2_pos['all_adjectives'], new_poem, cs_threshold, algo)
    new_poem, nouns_swapped = swapWords_new(poet_1_pos['nouns_poem'+ str(index)], poet_2_pos['all_nouns'], new_poem, cs_threshold, algo)
    swapped_table.extend(verbs_swapped)
    swapped_table.extend(adjectives_swapped)
    swapped_table.extend(nouns_swapped)
    return new_poem, swapped_table

In [ ]:
poem, table = newPoem_new(WS_pos, JWG_pos, WS_df.Poem_for_Human, 0, 0.6, 1)
getSemanticSimilarity(WS_df.Poem[0], poem, SentenceTransformer('stsb-roberta-large'))

0.9487894773483276

In [ ]:
poem, table = newPoem_new(WS_pos, JWG_pos, WS_df.Poem_for_Human, 0, 0.6, 2)
getSemanticSimilarity(WS_df.Poem[0], poem, SentenceTransformer('stsb-roberta-large'))

0.9487894773483276

## Topic Modeling

## LDA

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [ ]:
!python3 -m spacy download en

2023-11-03 05:38:22.437200: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-03 05:38:22.437272: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-03 05:38:22.437347: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-03 05:38:24.405430: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 40.3 MB/s eta 0:00:00
✔ Download and installation succe

In [ ]:
def choherence_perplexity_calculator(poem_list):
  data_words = list(sent_to_words(poem_list))
  print(data_words[:1])
  bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
  trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

  bigram_mod = gensim.models.phrases.Phraser(bigram)
  trigram_mod = gensim.models.phrases.Phraser(trigram)

  print(trigram_mod[bigram_mod[data_words[0]]])


  def remove_stopwords(texts):
      return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

  def make_bigrams(texts):
      return [bigram_mod[doc] for doc in texts]

  def make_trigrams(texts):
      return [trigram_mod[bigram_mod[doc]] for doc in texts]

  def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
      """https://spacy.io/api/annotation"""
      texts_out = []
      for sent in texts:
          doc = nlp(" ".join(sent))
          texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
      return texts_out
  # Remove Stop Words
  #data_words_nostops = remove_stopwords(data_words)

  # Form Bigrams
  data_words_bigrams = make_bigrams(data_words)

  # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
  # python3 -m spacy download en
  nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

  # Do lemmatization keeping only noun, adj, vb, adv
  data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN']) #, 'ADJ', 'VERB', 'ADV'

  print(data_lemmatized[:1])
  # Create Dictionary
  id2word = corpora.Dictionary(data_lemmatized)

  # Create Corpus
  texts = data_lemmatized

  # Term Document Frequency
  corpus = [id2word.doc2bow(text) for text in texts]

  # View
  print(corpus[:1])

  lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10,
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)
  print(lda_model.print_topics())
  # Visualize the topics
  pyLDAvis.enable_notebook()
  vis = gensimvis.prepare(lda_model, corpus, id2word)
  # vis
  # Compute Perplexity
  print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

  # Compute Coherence Score
  coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
  coherence_lda = coherence_model_lda.get_coherence()
  print('\nCoherence Score: ', coherence_lda)
  #return vis
  return lda_model, corpus, id2word

In [ ]:
WS_input = json.loads(WS_df.to_json(orient="index"))
JWG_input = json.loads(JWG_df.to_json(orient="index"))

In [ ]:
poem_count = len(WS_input)
WS_list=[]
for j in WS_input:
  poem_index =  str(j)
  WS_input[poem_index].pop('Poem_for_Human')
  WS_list.append(WS_input[poem_index])
print(WS_list)

[{'Unnamed: 0': 0, 'Title': 'The Sick Rose', 'Poem': 'rose thou art sick the invisible worm that flies in the night in the howling storm has found out thy bed of crimson joy and his dark secret love does thy life destroy'}, {'Unnamed: 0': 1, 'Title': "Love's Secret", 'Poem': 'never seek to tell thy love love that never told can be for the gentle wind does move silently invisibly told my love told my love told her all my heart trembling cold in ghastly fears ah she did depart soon as she was gone from me traveller came by silently invisibly he took her with sigh'}, {'Unnamed: 0': 2, 'Title': 'A Poison Tree', 'Poem': 'was angry with my friend told my wrath my wrath did end was angry with my foe told it not my wrath did grow and watered it in fears night and morning with my tears and sunned it with smiles and with soft deceitful wiles and it grew both day and night till it bore an apple bright and my foe beheld it shine and he knew that it was mine and into my garden stole when the night 

In [ ]:
poem_count = len(JWG_input)
JWG_list=[]
for j in JWG_input:
  poem_index =  str(j)
  JWG_input[poem_index].pop('Poem_for_Human')
  JWG_list.append(JWG_input[poem_index])
print(JWG_list)

[{'Unnamed: 0': 0, 'Title': 'Fire And Ice', 'Poem': 'some say the world will end in fire some say in ice from what have tasted of desire hold with those who favor fire but if it had to perish twice think know enough of hate to know that for destruction ice is also great and would suffice composition date is unknown the lyrical form of this poem is abaabcbcb'}, {'Unnamed: 0': 1, 'Title': 'Nothing Gold Can Stay', 'Poem': 'nature first green is gold her hardest hue to hold her early leaf flower but only so an hour then leaf subsides to leaf so eden sank to grief so dawn goes down to day nothing gold can stay'}, {'Unnamed: 0': 2, 'Title': 'Stopping By Woods On A Snowy Evening', 'Poem': 'whose woods these are think know his house is in the village though he will not see me stopping here to watch his woods fill up with snow my little horse must think it queer to stop without farmhouse near between the woods and frozen lake the darkest evening of the year he gives his harness bells shake to a

In [ ]:
lda_model, bow, diction = choherence_perplexity_calculator(WS_list)

[['unnamed', 'title', 'the', 'sick', 'rose', 'poem', 'rose', 'thou', 'art', 'sick', 'the', 'invisible', 'worm', 'that', 'flies', 'in', 'the', 'night', 'in', 'the', 'howling', 'storm', 'has', 'found', 'out', 'thy', 'bed', 'of', 'crimson', 'joy', 'and', 'his', 'dark', 'secret', 'love', 'does', 'thy', 'life', 'destroy']]
['unnamed_title', 'the', 'sick', 'rose', 'poem', 'rose', 'thou', 'art', 'sick', 'the', 'invisible', 'worm', 'that', 'flies', 'in', 'the', 'night', 'in', 'the', 'howling', 'storm', 'has', 'found', 'out', 'thy', 'bed', 'of', 'crimson', 'joy', 'and', 'his', 'dark', 'secret', 'love', 'does', 'thy', 'life', 'destroy']
[['rose', 'poem', 'art', 'worm', 'night', 'storm', 'bed', 'joy', 'love', 'life']]
[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1)]]
[(0, '0.043*"summer" + 0.043*"joy" + 0.022*"tender" + 0.022*"morn" + 0.022*"day" + 0.022*"bird" + 0.012*"fear" + 0.012*"thro" + 0.012*"eye" + 0.012*"book"'), (1, '0.004*"joy" + 0.004*"summer" + 0.003*

In [ ]:
gensimvis.prepare(lda_model, bow, diction)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.111988  0.042144       1        1  35.163450
4     -0.166846 -0.036936       2        1  20.367904
2     -0.118520  0.073072       3        1  14.563502
7      0.041778 -0.122317       4        1   9.074170
0      0.038302  0.113657       5        1   7.407631
9      0.019839 -0.035428       6        1   5.221325
8      0.014356 -0.011480       7        1   4.661865
6      0.025644 -0.025497       8        1   3.015716
1      0.018193  0.004829       9        1   0.262218
5      0.015266 -0.002044      10        1   0.262218, topic_info=       Term       Freq      Total Category  logprob  loglift
4      love  10.000000  10.000000  Default  30.0000  30.0000
6      poem   9.000000   9.000000  Default  29.0000  29.0000
167  summer   5.000000   5.000000  Default  28.0000  28.0000
2       joy   9.000000   9.000000  Default  27.0000  27.0000
33    child   5.000000   5.000000  Default  26.0000  26.0000
..      ...        ...        ...      ...      ...      ...
94      woe   0.004987   3.807336  Topic10  -5.9474  -0.6940
33    child   0.005004   5.868211  Topic10  -5.9440  -1.1232
207  flower   0.004998   7.497375  Topic10  -5.9452  -1.3695
50      eye   0.004976   4.973671  Topic10  -5.9497  -0.9635
155    morn   0.004953   3.995438  Topic10  -5.9542  -0.7491

[533 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
209       1  0.732671     age
210       1  0.579930     air
104       7  1.047875  albion
105       7  1.047875   altar
45        3  0.820606   anvil
...     ...       ...     ...
32        1  0.272423   wrath
32        2  0.544846   wrath
171       5  0.935970    year
139       6  1.025927   youth
128       7  1.047875    zeal

[402 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 5, 3, 8, 1, 10, 9, 7, 2, 6])

In [ ]:
lda_model, bow, diction = choherence_perplexity_calculator(JWG_list)

[['unnamed', 'title', 'fire', 'and', 'ice', 'poem', 'some', 'say', 'the', 'world', 'will', 'end', 'in', 'fire', 'some', 'say', 'in', 'ice', 'from', 'what', 'have', 'tasted', 'of', 'desire', 'hold', 'with', 'those', 'who', 'favor', 'fire', 'but', 'if', 'it', 'had', 'to', 'perish', 'twice', 'think', 'know', 'enough', 'of', 'hate', 'to', 'know', 'that', 'for', 'destruction', 'ice', 'is', 'also', 'great', 'and', 'would', 'suffice', 'composition', 'date', 'is', 'unknown', 'the', 'lyrical', 'form', 'of', 'this', 'poem', 'is', 'abaabcbcb']]
['unnamed_title', 'fire', 'and', 'ice', 'poem', 'some', 'say', 'the', 'world', 'will', 'end', 'in', 'fire', 'some', 'say', 'in', 'ice', 'from', 'what', 'have', 'tasted', 'of', 'desire', 'hold', 'with', 'those', 'who', 'favor', 'fire', 'but', 'if', 'it', 'had', 'to', 'perish', 'twice', 'think', 'know', 'enough', 'of', 'hate', 'to', 'know', 'that', 'for', 'destruction', 'ice', 'is', 'also', 'great', 'and', 'would', 'suffice', 'composition', 'date', 'is', 'un

In [ ]:
gensimvis.prepare(lda_model, bow, diction)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.112107  0.154850       1        1  28.155822
9      0.124685 -0.116769       2        1  14.698283
3      0.015118  0.094004       3        1  13.900458
1     -0.179428  0.043019       4        1  11.019266
8      0.011628 -0.023819       5        1   7.346565
7     -0.038496 -0.029283       6        1   7.203357
0     -0.029740 -0.085998       7        1   6.387351
6     -0.011919 -0.026656       8        1   5.716575
2      0.012410 -0.013247       9        1   5.310317
4     -0.016364  0.003898      10        1   0.262007, topic_info=       Term      Freq     Total Category  logprob  loglift
232   apple  6.000000  6.000000  Default  30.0000  30.0000
14   flower  7.000000  7.000000  Default  29.0000  29.0000
39     wood  8.000000  8.000000  Default  28.0000  28.0000
33    sleep  5.000000  5.000000  Default  27.0000  27.0000
103     boy  8.000000  8.000000  Default  26.0000  26.0000
..      ...       ...       ...      ...      ...      ...
39     wood  0.004897  8.490338  Topic10  -5.9744  -1.5135
327  winter  0.004897  2.925556  Topic10  -5.9744  -0.4480
78      eye  0.004897  4.127772  Topic10  -5.9745  -0.7924
63     life  0.004896  4.004875  Topic10  -5.9748  -0.7624
351  matter  0.004894  1.412205  Topic10  -5.9750   0.2797

[479 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
0         3  0.783758  abaabcbcb
297       4  0.803657       ache
100       2  0.769089     affair
381       9  0.935063  aftermath
41        3  0.678907        age
...     ...       ...        ...
221       6  0.869055         ye
40        1  0.398461       year
40        5  0.398461       year
40        8  0.398461       year
92        3  0.783758  yesterday

[409 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 10, 4, 2, 9, 8, 1, 7, 3, 5])

In [ ]:
def ner(text):
  return spacy.load("en_core_web_sm")(text)

In [ ]:
original = ner(WS_input['0']['Poem'])

In [ ]:
displacy.render(original, style="ent", jupyter=True)

## BERTopic

In [ ]:
def remove_stopwords(texts):
    stopword_list = nltk.corpus.stopwords.words('english')
    return [[word for word in simple_preprocess(str(doc)) if word not in stopword_list] for doc in texts]

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(WS_df.Poem.to_list() + JWG_df.Poem.to_list()))

print(data_words[:1])
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

[['rose', 'thou', 'art', 'sick', 'the', 'invisible', 'worm', 'that', 'flies', 'in', 'the', 'night', 'in', 'the', 'howling', 'storm', 'has', 'found', 'out', 'thy', 'bed', 'of', 'crimson', 'joy', 'and', 'his', 'dark', 'secret', 'love', 'does', 'thy', 'life', 'destroy']]


In [ ]:
docs = WS_df.Poem.to_list() + JWG_df.Poem.to_list()

In [ ]:
docs = remove_stopwords(docs)
docs = make_bigrams(docs)
docs = make_trigrams(docs)
docs = lemmatization(docs)

In [ ]:
docs=[[' '.join(i)] for i in docs]
newDocs = list()
for i in docs:
    for j in i:
        newDocs.append(j)
print(newDocs)

['rise thou art sick invisible worm fly night howl storm find bed secret love life destroy', 'never seek tell love love never tell gentle wind move silently invisibly tell love tell love tell heart tremble cold ghastly fear depart soon go traveller come silently invisibly take sigh', 'angry friend tell end angry foe tell grow water fear night morning tear sun smile soft deceitful wile grow day night bore apple bright shine know mine garden stole night veil pole morning glad see foe outstretche tree', 'little lamb make dost thou know make give life bid feed stream mead give clothing delight soft clothing wooly give tender voice make vale rejoice little lamb make dost thou know make little tell little tell call name call mild become little child child call name little bless little bless', 'tyger tyger burn bright forest night immortal hand eye frame fearful symmetry distant deep sky burn fire thine eye wing dare aspire hand dare seize fire shoulder art twist sinew heart heart begin beat 

In [ ]:
model = BERTopic(embedding_model=SentenceTransformer('stsb-roberta-large'))

topics, probabilities = model.fit_transform(newDocs)

In [ ]:
model.get_topic_freq().head(11)

,Topic,Count
0,-1,40


In [ ]:
model.get_topic(-1)

[('go', 0.055551237388084714),
 ('see', 0.04612333892434992),
 ('say', 0.04612333892434992),
 ('make', 0.04472254001553938),
 ('love', 0.04042700973386802),
 ('know', 0.04042700973386802),
 ('night', 0.03896179184666121),
 ('little', 0.03747855796133477),
 ('come', 0.03747855796133477),
 ('tree', 0.03747855796133477)]

##NNMF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

# Convert your preprocessed text data into a TF-IDF matrix
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(newDocs)

# Specify the number of topics you want to extract
num_topics = 10  # You can adjust this as needed

# Fit the NMF model to your TF-IDF matrix
nmf = NMF(n_components=num_topics, random_state=1)
nmf.fit(tfidf)

# Get the top words for each topic
def get_top_words(model, feature_names, n_top_words):
    topics = []
    for topic_idx, topic in enumerate(model.components_):
        top_words_idx = topic.argsort()[:-n_top_words - 1:-1]
        top_words = [feature_names[i] for i in top_words_idx]
        topics.append(top_words)
    return topics

n_top_words = 10  # You can adjust this as needed
feature_names = tfidf_vectorizer.get_feature_names_out()
top_words_per_topic = get_top_words(nmf, feature_names, n_top_words)

# Print the top words for each topic
for i, topic in enumerate(top_words_per_topic):
    print(f"Topic {i + 1}: {', '.join(topic)}")

Topic 1: ice, say, date, wall, know, poem, think, make, composition, lyrical
Topic 2: thorn, flower, rose, delight, tend, love, tree, bore, beauty, bright
Topic 3: little, weep, love, weeping, make, child, black, priest, hear, vain
Topic 4: tell, night, apple, morning, grow, fear, sleep, love, glad, seek
Topic 5: laugh, sing, joy, bird, summer, sit, spring, morn, sweet, green
Topic 6: hand, fly, art, night, life, eye, boy, burn, thou, storm
Topic 7: blow, snow, rain, forget, day, step, old, bring, tool, turn
Topic 8: word, leaf, stand, doubt, dead, garden, come, carry, flower, rattle
Topic 9: age, wood, travel, way, road, know, long, bid, happy, look
Topic 10: human, infant, forge, face, bear, man, soul, hear, woe, tear


#NNMF FOR POEMS

In [ ]:
def preprocess_data(poem_list):
    data_words = list(sent_to_words(poem_list))
    bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
    trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)

    data_words_bigrams = make_bigrams(data_words)

    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN'])

    return [' '.join(text) for text in data_lemmatized]

In [ ]:
def nmf_topic_modeling(data, n_topics=10):
    # Convert data to TF-IDF matrix
    vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
    tfidf = vectorizer.fit_transform(data)

    # Apply NNMF
    nmf_model = NMF(n_components=n_topics, random_state=1, l1_ratio=.5, init='nndsvd').fit(tfidf)

    # Display topics
    feature_names = vectorizer.get_feature_names_out()
    topics = []
    for topic_idx, topic in enumerate(nmf_model.components_):
        top_words = " ".join([feature_names[i] for i in topic.argsort()[:-10 - 1:-1]])
        topics.append(f"Topic {topic_idx + 1}: {top_words}")

    return topics

In [ ]:
processed_WS_data = preprocess_data(WS_list)
WS_topics = nmf_topic_modeling(processed_WS_data)
print("\nTopics for WS dataset:")
for topic in WS_topics:
    print(topic)


Topics for WS dataset:
Topic 1: joy bird summer air run shade stream song voice morn
Topic 2: love rose horn delight poem garden traveller joy sigh fear
Topic 3: flower tree night morning wrath day earth fear morn thorn
Topic 4: hand fly eye art wing summer night death man innocence
Topic 5: worm bed night shade emmet angel sigh tear father life
Topic 6: street infant woe chimney sweeper church harlot curse blood soldier
Topic 7: weeping child boy hair door priest father thing nought shore
Topic 8: form heart face furnace terror jealousy iron poem eye forest
Topic 9: snow desire time traveller grave sun poem woe heath winter
Topic 10: thou clothing child stream lamb voice life soul mother light


In [ ]:
processed_JWG_data = preprocess_data(JWG_list)
JWG_topics = nmf_topic_modeling(processed_JWG_data)
print("\nTopics for JWG dataset:")
for topic in JWG_topics:
    print(topic)


Topics for JWG dataset:
Topic 1: snow day dust rain tree change heart poem way love
Topic 2: bird garden flower wall thought tree half end moment word
Topic 3: age way wood day step morning leave pleasure theme village
Topic 4: ice date composition form poem ground world tree trunk line
Topic 5: word time moment air nature wind work knee door hill
Topic 6: sleep wood apple sound year wind village house load world
Topic 7: gold flower hour nature garden sun way day poem child
Topic 8: boy hand heart work half wall breeze dust load day
Topic 9: night rain lane sky eye foot light time sound house
Topic 10: step sense end poem garden apple way half morning word


##Statistics

In [ ]:
print('Verbs in William Blake poem 1', len(WS_pos['verbs_poem0']))
print('Nouns in William Blake poem 1', len(WS_pos['nouns_poem0']))
print('Adjectives in William Blake poem 1', len(WS_pos['adjectives_poem0']))
print('Adverbs in William Blake poem 1', len(WS_pos['adverbs_poem0']))

Verbs in William Blake poem 1 6
Nouns in William Blake poem 1 8
Adjectives in William Blake poem 1 6
Adverbs in William Blake poem 1 0


In [ ]:
print('Verbs in William Blake poem 2', len(WS_pos['verbs_poem1']))
print('Nouns in William Blake poem 2', len(WS_pos['nouns_poem1']))
print('Adjectives in William Blake poem 2', len(WS_pos['adjectives_poem1']))
print('Adverbs in William Blake poem 2', len(WS_pos['adverbs_poem1']))

Verbs in William Blake poem 2 9
Nouns in William Blake poem 2 7
Adjectives in William Blake poem 2 1
Adverbs in William Blake poem 2 5


In [ ]:
print('Verbs in Robert Frost poem 1', len(JWG_pos['verbs_poem0']))
print('Nouns in Robert Frost poem 1', len(JWG_pos['nouns_poem0']))
print('Adjectives in Robert Frost poem 1', len(JWG_pos['adjectives_poem0']))
print('Adverbs in Robert Frost poem 1', len(JWG_pos['adverbs_poem0']))

Verbs in Robert Frost poem 1 10
Nouns in Robert Frost poem 1 11
Adjectives in Robert Frost poem 1 4
Adverbs in Robert Frost poem 1 2


In [ ]:
print('Verbs in Robert Frost poem 2', len(JWG_pos['verbs_poem1']))
print('Nouns in Robert Frost poem 2', len(JWG_pos['nouns_poem1']))
print('Adjectives in Robert Frost poem 2', len(JWG_pos['adjectives_poem1']))
print('Adverbs in Robert Frost poem 2', len(JWG_pos['adverbs_poem1']))

Verbs in Robert Frost poem 2 4
Nouns in Robert Frost poem 2 10
Adjectives in Robert Frost poem 2 3
Adverbs in Robert Frost poem 2 5
